In [1]:
import torch
import torch.nn as nn
import math

In [ ]:
# Define the ModelArgs configuration class
from dataclasses import dataclass

@dataclass
class ModelArgs:
    """模型配置参数类"""
    dim: int = 512                    # 模型维度
    n_heads: int = 8                  # 注意力头数
    n_layers: int = 6                 # 层数
    vocab_size: int = 50000           # 词汇表大小
    max_seq_len: int = 2048           # 最大序列长度
    dropout: float = 0.1              # Dropout 比率
    bias: bool = False                # 是否使用偏置

print("ModelArgs class defined successfully!")
print("Available parameters:")
for field_name in ModelArgs.__dataclass_fields__:
    field = ModelArgs.__dataclass_fields__[field_name]
    print(f"  {field_name}: {field.type.__name__} = {field.default}")


In [14]:
'''注意力计算函数'''
def attention(query, key, value, dropout=None):
    '''
    计算"缩放点积注意力"。

    参数:
    query: 查询张量，形状为 (..., seq_len_q, d_k)
    key: 键张量，形状为 (..., seq_len_k, d_k)
    value: 值张量，形状为 (..., seq_len_v, d_v)，通常 seq_len_k == seq_len_v
    dropout: Dropout 层，可选

    返回:
    加权后的值张量，形状为 (..., seq_len_q, d_v)
    注意力权重张量，形状为 (..., seq_len_q, seq_len_k)
    '''
    # 获取键向量的维度 d_k
    d_k = query.size(-1)
    print(f"d_k: {d_k}")
    
    # 计算 Q 与 K 的转置的点积，然后缩放
    # 注意力得分: scores = Q × K^T / √d_k
    # scores 的形状: (..., seq_len_q, seq_len_k)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    
    # 对 scores 的最后一个维度应用 softmax，得到注意力权重
    # p_attn 的形状: (..., seq_len_q, seq_len_k)
    p_attn = scores.softmax(dim=-1)
    
    # (可选) 应用 dropout
    if dropout is not None:
        p_attn = dropout(p_attn)
        
    # 将注意力权重 p_attn 与 V 相乘，得到加权的输出
    # output 的形状: (..., seq_len_q, d_v)
    return torch.matmul(p_attn, value), p_attn

In [ ]:
# Example to execute
def attention_test():
    # 设定超参数
    batch_size = 2      # 批处理大小
    seq_len = 5         # 序列长度
    d_k = 8             # Query 和 Key 的维度
    d_v = 10            # Value 的维度 (可以和 d_k 不同)
    dropout_rate = 0.1  # Dropout 比率

    # 1. 实例化 Dropout 层
    dropout_layer = nn.Dropout(p=dropout_rate)

    # 2. 创建模拟的 Q, K, V 张量
    # 形状: (batch_size, seq_len, dimension)
    query = torch.randn(batch_size, seq_len, d_k)
    key = torch.randn(batch_size, seq_len, d_k)
    value = torch.randn(batch_size, seq_len, d_v)

    print("--- 输入张量形状 ---")
    print(f"Query (Q) shape: {query.shape}")
    print(f"Key (K) shape:   {key.shape}")
    print(f"Value (V) shape: {value.shape}")
    print("-" * 20)

    # 3. 调用 attention 函数
    # 将 dropout_layer 作为参数传入
    output, attention_weights = attention(query, key, value, dropout=dropout_layer)

    # 4. 打印输出结果的形状
    print("\n--- 输出结果形状 ---")
    print(f"Output shape: {output.shape}")
    print(f"Attention Weights shape: {attention_weights.shape}")
    print("-" * 20)

    # 5. 检查输出内容的含义
    print("\n--- 输出内容检查 ---")
    print("Output[0, 0, :]:\n", output[0, 0, :])
    print("\n第一个样本的注意力权重 (部分):\n", attention_weights[0].round(decimals=4))
    # 检查注意力权重的和是否为1 (在最后一个维度上)
    sum_of_weights = attention_weights[0, 0, :].sum()
    print(f"\n第一行注意力权重的和: {sum_of_weights:.4f} (应约等于 1.0)")

In [16]:
attention_test()

--- 输入张量形状 ---
Query (Q) shape: torch.Size([2, 5, 8])
Key (K) shape:   torch.Size([2, 5, 8])
Value (V) shape: torch.Size([2, 5, 10])
--------------------
d_k: 8

--- 输出结果形状 ---
Output shape: torch.Size([2, 5, 10])
Attention Weights shape: torch.Size([2, 5, 5])
--------------------

--- 输出内容检查 ---
Output[0, 0, :]:
 tensor([ 0.9657,  0.2202, -0.5204, -0.5764, -1.5707, -0.3773, -0.9463,  1.3893,
         1.2438,  0.4634])

第一个样本的注意力权重 (部分):
 tensor([[0.0000, 0.1387, 0.0465, 0.0915, 0.7841],
        [0.3416, 0.0135, 0.0407, 0.6249, 0.0904],
        [0.1342, 0.0799, 0.0927, 0.2342, 0.5700],
        [0.2875, 0.0857, 0.0203, 0.3677, 0.3498],
        [0.2472, 0.2715, 0.1490, 0.3754, 0.0680]])

第一行注意力权重的和: 1.0608 (应约等于 1.0)


In [23]:
def self_attention_test():
    # self attention 是可以将相同的 Q, K, V 传入attention机制里
    batch_size = 2      # 批处理大小
    seq_len = 5         # 序列长度
    d_k = 8             # Query 和 Key 的维度，因为我们是自注意力，因此这也是Value的维度
    dropout_rate = 0.1  # Dropout 比率

    # 1. 实例化 Dropout 层
    dropout_layer = nn.Dropout(p=dropout_rate)
    
    x = torch.randn(batch_size, seq_len, d_k)
    print(f"x.shape: {x.shape}")
    
    # 2. 计算自注意力
    output, p_attn = attention(x, x, x, dropout_layer)
    
    # 3. 打印输出形状
    print(f"Output shape: {output.shape}")
    print(f"Attention weights shape: {p_attn.shape}")
    
    return output, p_attn
    

In [24]:
self_attention_test()

x.shape: torch.Size([2, 5, 8])
d_k: 8
Output shape: torch.Size([2, 5, 8])
Attention weights shape: torch.Size([2, 5, 5])


(tensor([[[-0.6683,  0.8487,  0.3628, -1.3097, -0.8950,  0.6976, -1.5617,
           -0.6750],
          [ 2.4377,  0.0092, -0.2172,  1.1283,  0.5985, -2.2264,  1.5915,
           -0.6868],
          [-0.8538,  0.0929,  0.2378, -0.3035,  1.2549,  0.0255,  0.4164,
            0.3867],
          [ 1.5358,  0.2174,  0.0970, -0.2330,  1.0490, -0.1617,  1.1028,
            0.4111],
          [ 1.0493,  0.3808, -0.1211, -0.8172,  0.7783,  0.2532,  0.8740,
            0.6752]],
 
         [[-0.4442, -0.1237, -0.1255, -0.0936,  0.2974,  0.1481, -0.6693,
            0.5486],
          [-0.4099,  0.6186,  0.3617, -0.8760, -0.7344, -0.4572,  0.2512,
           -0.3626],
          [-1.2514, -0.2184, -1.0676,  0.4270,  0.5719, -0.3268, -2.2355,
            1.5484],
          [-0.4894,  0.0703,  0.0202, -0.4927, -0.7036, -0.4252,  0.3350,
           -0.3648],
          [-0.2227, -0.7650,  0.9707, -1.1396,  1.5226,  1.9492, -0.5938,
            0.7727]]]),
 tensor([[[9.9057e-01, 5.8785e-03, 4.4457e-0

In [32]:
import torch.nn.functional as F
import torch.nn as nn
import math
from dataclasses import dataclass

@dataclass
class ModelArgs:
    dim: int = 512                    # 模型维度
    n_heads: int = 8                  # 注意力头数
    dropout: float = 0.1              # Dropout 比率
    max_seq_len: int = 2048           # 最大序列长度

# 修复 MultiHeadAttention 类中的一个小错误
'''多头自注意力计算模块'''
class MultiHeadAttention(nn.Module):

    def __init__(self, args: ModelArgs, is_causal=False):
        # 构造函数
        # args: 配置对象
        super().__init__()
        # 隐藏层维度必须是头数的整数倍，因为后面我们会将输入拆成头数个矩阵
        assert args.dim % args.n_heads == 0
        # 模型并行处理大小，默认为1。
        model_parallel_size = 1
        # 本地计算头数，等于总头数除以模型并行处理大小。
        self.n_local_heads = args.n_heads // model_parallel_size
        # 每个头的维度，等于模型维度除以头的总数。
        self.head_dim = args.dim // args.n_heads
        # 保存是否使用因果掩码
        self.is_causal = is_causal

        # Wq, Wk, Wv 参数矩阵，每个参数矩阵为 n_embd x n_embd
        # 这里通过三个组合矩阵来代替了n个参数矩阵的组合，其逻辑在于矩阵内积再拼接其实等同于拼接矩阵再内积，
        # 不理解的读者可以自行模拟一下，每一个线性层其实相当于n个参数矩阵的拼接
        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wk = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wv = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        # 输出权重矩阵，维度为 dim x n_embd（head_dim = n_embeds / n_heads）
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)
        # 注意力的 dropout
        self.attn_dropout = nn.Dropout(args.dropout)
        # 残差连接的 dropout
        self.resid_dropout = nn.Dropout(args.dropout)
         
        # 创建一个上三角矩阵，用于遮蔽未来信息
        # 注意，因为是多头注意力，Mask 矩阵比之前我们定义的多一个维度
        if is_causal:
           mask = torch.full((1, 1, args.max_seq_len, args.max_seq_len), float("-inf"))
           mask = torch.triu(mask, diagonal=1)
           # 注册为模型的缓冲区
           self.register_buffer("mask", mask)

    def forward(self, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor):

        # 获取批次大小和序列长度，[batch_size, seq_len, dim]
        bsz, seqlen, _ = q.shape

        # 计算查询（Q）、键（K）、值（V）,输入通过参数矩阵层，维度为 (B, T, n_embed) x (n_embed, n_embed) -> (B, T, n_embed)
        xq, xk, xv = self.wq(q), self.wk(k), self.wv(v)
        print(f"After linear transformation:")
        print(f"  xq shape: {xq.shape}")
        print(f"  xk shape: {xk.shape}")
        print(f"  xv shape: {xv.shape}")

        # 将 Q、K、V 拆分成多头，维度为 (B, T, n_head, C // n_head)，然后交换维度，变成 (B, n_head, T, C // n_head)
        # 因为在注意力计算中我们是取了后两个维度参与计算
        # 为什么要先按B*T*n_head*C//n_head展开再互换1、2维度而不是直接按注意力输入展开，是因为view的展开方式是直接把输入全部排开，
        # 然后按要求构造，可以发现只有上述操作能够实现我们将每个头对应部分取出来的目标
        xq = xq.view(bsz, seqlen, self.n_local_heads, self.head_dim)
        xk = xk.view(bsz, seqlen, self.n_local_heads, self.head_dim)
        xv = xv.view(bsz, seqlen, self.n_local_heads, self.head_dim)
        print(f"After view (reshape):")
        print(f"  xq shape: {xq.shape}")
        print(f"  xk shape: {xk.shape}")
        print(f"  xv shape: {xv.shape}")
        
        xq = xq.transpose(1, 2)
        xk = xk.transpose(1, 2)
        xv = xv.transpose(1, 2)
        print(f"After transpose:")
        print(f"  xq shape: {xq.shape}")
        print(f"  xk shape: {xk.shape}")
        print(f"  xv shape: {xv.shape}")


        # 注意力计算
        # 计算 QK^T / sqrt(d_k)，维度为 (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        scores = torch.matmul(xq, xk.transpose(2, 3)) / math.sqrt(self.head_dim)
        print(f"Attention scores shape: {scores.shape}")
        
        # 掩码自注意力必须有注意力掩码
        if self.is_causal:
            assert hasattr(self, 'mask')
            # 这里截取到序列长度，因为有些序列可能比 max_seq_len 短
            scores = scores + self.mask[:, :, :seqlen, :seqlen]
            print(f"After applying causal mask")
            
        # 计算 softmax，维度为 (B, nh, T, T)
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)
        # 做 Dropout
        scores = self.attn_dropout(scores)
        print(f"Attention weights shape: {scores.shape}")
        
        # V * Score，维度为(B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        output = torch.matmul(scores, xv)
        print(f"After attention output shape: {output.shape}")

        # 恢复时间维度并合并头。
        # 将多头的结果拼接起来, 先交换维度为 (B, T, n_head, C // n_head)，再拼接成 (B, T, n_head * C // n_head)
        # contiguous 函数用于重新开辟一块新内存存储，因为Pytorch设置先transpose再view会报错，
        # 因为view直接基于底层存储得到，然而transpose并不会改变底层存储，因此需要额外存储
        output = output.transpose(1, 2).contiguous().view(bsz, seqlen, -1)
        print(f"After merging heads shape: {output.shape}")

        # 最终投影回残差流。
        output = self.wo(output)
        output = self.resid_dropout(output)
        print(f"Final output shape: {output.shape}")
        return output


In [33]:
def multi_head_attention_demo():
    """
    多头注意力机制的完整演示
    """
    print("=" * 50)
    print("多头注意力机制演示")
    print("=" * 50)
    
    # 1. 创建配置参数
    args = ModelArgs(
        dim=512,         # 模型维度
        n_heads=8,       # 8个注意力头
        dropout=0.1,     # dropout率
        max_seq_len=2048 # 最大序列长度
    )
    
    print(f"配置参数:")
    print(f"  模型维度 (dim): {args.dim}")
    print(f"  注意力头数 (n_heads): {args.n_heads}")
    print(f"  每个头的维度 (head_dim): {args.dim // args.n_heads}")
    print(f"  dropout率: {args.dropout}")
    print()
    
    # 2. 创建多头注意力模块
    # 我们创建两个版本：一个普通的，一个带因果掩码的
    print("创建两个多头注意力模块:")
    mha_normal = MultiHeadAttention(args, is_causal=False)
    mha_causal = MultiHeadAttention(args, is_causal=True)
    print("  - 普通多头注意力 (双向)")
    print("  - 因果多头注意力 (单向，用于解码器)")
    print()
    
    # 3. 创建输入数据
    batch_size = 2
    seq_len = 10
    
    # 创建随机输入张量
    x = torch.randn(batch_size, seq_len, args.dim)
    print(f"输入数据:")
    print(f"  输入形状: {x.shape}")
    print(f"  含义: [batch_size={batch_size}, seq_len={seq_len}, dim={args.dim}]")
    print()
    
    # 4. 演示普通多头注意力 (自注意力)
    print("=" * 30)
    print("普通多头注意力 (自注意力)")
    print("=" * 30)
    
    # 设置为评估模式以避免随机性
    mha_normal.eval()
    with torch.no_grad():
        output_normal = mha_normal(x, x, x)  # 自注意力：Q=K=V=x
        
    print(f"\\n输出结果:")
    print(f"  输出形状: {output_normal.shape}")
    print(f"  输出是否等于输入形状: {output_normal.shape == x.shape}")
    print()
    
    # 5. 演示因果多头注意力
    print("=" * 30)
    print("因果多头注意力 (带掩码)")
    print("=" * 30)
    
    mha_causal.eval()
    with torch.no_grad():
        output_causal = mha_causal(x, x, x)  # 因果自注意力
        
    print(f"\\n输出结果:")
    print(f"  输出形状: {output_causal.shape}")
    print(f"  输出是否等于输入形状: {output_causal.shape == x.shape}")
    print()
    
    # 6. 比较两个输出
    print("=" * 30)
    print("输出比较")
    print("=" * 30)
    
    # 计算输出的差异
    diff = torch.abs(output_normal - output_causal).mean()
    print(f"普通注意力与因果注意力输出的平均绝对差异: {diff:.6f}")
    print("(差异应该很大，因为因果注意力只能看到过去的信息)")
    
    return output_normal, output_causal

# 运行演示
multi_head_attention_demo()


多头注意力机制演示
配置参数:
  模型维度 (dim): 512
  注意力头数 (n_heads): 8
  每个头的维度 (head_dim): 64
  dropout率: 0.1

创建两个多头注意力模块:
  - 普通多头注意力 (双向)
  - 因果多头注意力 (单向，用于解码器)

输入数据:
  输入形状: torch.Size([2, 10, 512])
  含义: [batch_size=2, seq_len=10, dim=512]

普通多头注意力 (自注意力)
After linear transformation:
  xq shape: torch.Size([2, 10, 512])
  xk shape: torch.Size([2, 10, 512])
  xv shape: torch.Size([2, 10, 512])
After view (reshape):
  xq shape: torch.Size([2, 10, 8, 64])
  xk shape: torch.Size([2, 10, 8, 64])
  xv shape: torch.Size([2, 10, 8, 64])
After transpose:
  xq shape: torch.Size([2, 8, 10, 64])
  xk shape: torch.Size([2, 8, 10, 64])
  xv shape: torch.Size([2, 8, 10, 64])
Attention scores shape: torch.Size([2, 8, 10, 10])
Attention weights shape: torch.Size([2, 8, 10, 10])
After attention output shape: torch.Size([2, 8, 10, 64])
After merging heads shape: torch.Size([2, 10, 512])
Final output shape: torch.Size([2, 10, 512])
\n输出结果:
  输出形状: torch.Size([2, 10, 512])
  输出是否等于输入形状: True

因果多头注意力 (带掩码)
After li

(tensor([[[-3.5882e-03,  3.4845e-01, -1.5955e-01,  ..., -5.2484e-02,
           -5.0627e-02, -4.3429e-02],
          [-4.2136e-02,  2.9440e-01, -2.5072e-01,  ..., -6.8263e-02,
            6.0080e-02,  9.8534e-03],
          [-4.6872e-02,  2.9721e-01, -1.5375e-01,  ..., -3.9070e-02,
           -1.0439e-02, -1.0417e-02],
          ...,
          [ 2.0015e-02,  3.5448e-01, -2.0549e-01,  ..., -6.1390e-02,
           -4.2314e-02, -4.7186e-05],
          [-1.0270e-01,  3.2186e-01, -2.1751e-01,  ..., -4.9039e-02,
            1.4546e-02,  4.3820e-02],
          [-3.0241e-02,  2.6220e-01, -2.4302e-01,  ..., -1.0419e-01,
            5.5745e-02, -6.3002e-02]],
 
         [[-2.1459e-02,  3.7747e-02, -5.3677e-02,  ...,  1.2905e-01,
           -2.2530e-01,  1.3839e-01],
          [-1.1947e-01,  6.3499e-02, -1.5479e-01,  ...,  8.2109e-02,
           -2.2436e-01,  1.0165e-01],
          [-4.5769e-02,  8.1421e-02, -1.5156e-01,  ...,  9.5839e-02,
           -2.2262e-01,  1.0562e-01],
          ...,
    

In [35]:
def detailed_dimension_demo():
    """
    详细的维度变化演示 - 使用小规模数据便于理解
    """
    print("=" * 60)
    print("详细维度变化演示")
    print("=" * 60)
    
    # 使用小规模参数便于理解
    args = ModelArgs(
        dim=64,          # 小的模型维度
        n_heads=4,       # 4个注意力头
        dropout=0.0,     # 不使用dropout便于演示
        max_seq_len=10   # 小的最大序列长度
    )
    
    print(f"小规模配置参数:")
    print(f"  模型维度 (dim): {args.dim}")
    print(f"  注意力头数 (n_heads): {args.n_heads}")
    print(f"  每个头的维度 (head_dim): {args.dim // args.n_heads}")
    print()
    
    # 创建多头注意力模块
    mha = MultiHeadAttention(args, is_causal=False)
    mha.eval()
    
    # 创建小规模输入数据
    batch_size = 1  # 单个批次
    seq_len = 3     # 3个词的序列
    
    x = torch.randn(batch_size, seq_len, args.dim)
    print(f"输入数据:")
    print(f"  输入形状: {x.shape}")
    print(f"  含义: [batch_size={batch_size}, seq_len={seq_len}, dim={args.dim}]")
    print(f"  这表示：1个样本，3个词，每个词64维特征")
    print()
    
    print("=" * 40)
    print("逐步执行前向传播...")
    print("=" * 40)
    
    with torch.no_grad():
        output = mha(x, x, x)  # 自注意力
        
    print("\\n" + "=" * 40)
    print("总结")
    print("=" * 40)
    print(f"最终输出形状: {output.shape}")
    print(f"输入输出形状一致: {output.shape == x.shape}")
    print(f"这意味着每个词仍然是64维特征，但现在包含了其他词的信息")
    
    return output

# 运行详细演示
detailed_dimension_demo()


详细维度变化演示
小规模配置参数:
  模型维度 (dim): 64
  注意力头数 (n_heads): 4
  每个头的维度 (head_dim): 16

输入数据:
  输入形状: torch.Size([1, 3, 64])
  含义: [batch_size=1, seq_len=3, dim=64]
  这表示：1个样本，3个词，每个词64维特征

逐步执行前向传播...
After linear transformation:
  xq shape: torch.Size([1, 3, 64])
  xk shape: torch.Size([1, 3, 64])
  xv shape: torch.Size([1, 3, 64])
After view (reshape):
  xq shape: torch.Size([1, 3, 4, 16])
  xk shape: torch.Size([1, 3, 4, 16])
  xv shape: torch.Size([1, 3, 4, 16])
After transpose:
  xq shape: torch.Size([1, 4, 3, 16])
  xk shape: torch.Size([1, 4, 3, 16])
  xv shape: torch.Size([1, 4, 3, 16])
Attention scores shape: torch.Size([1, 4, 3, 3])
Attention weights shape: torch.Size([1, 4, 3, 3])
After attention output shape: torch.Size([1, 4, 3, 16])
After merging heads shape: torch.Size([1, 3, 64])
Final output shape: torch.Size([1, 3, 64])
\n========================================
总结
最终输出形状: torch.Size([1, 3, 64])
输入输出形状一致: True
这意味着每个词仍然是64维特征，但现在包含了其他词的信息


tensor([[[ 0.1248, -0.4088, -0.1043, -0.1492, -0.1853, -0.1101, -0.0196,
          -0.1588, -0.3738,  0.1675, -0.1530,  0.2170, -0.2289,  0.2016,
           0.1170, -0.0964,  0.2748, -0.1827,  0.0985,  0.0850, -0.0833,
           0.0658,  0.2059, -0.0165,  0.0487, -0.2841, -0.1550, -0.0300,
           0.2595,  0.2782, -0.2102,  0.0638, -0.1863,  0.1755,  0.0742,
           0.0574,  0.1980, -0.0122, -0.1676,  0.1484,  0.2180, -0.3638,
           0.2751, -0.0568,  0.0623,  0.1186, -0.4737,  0.0195, -0.0804,
          -0.1568,  0.2000,  0.1656, -0.0141, -0.1310,  0.0199, -0.0208,
          -0.0608,  0.3282,  0.2732, -0.0519, -0.0837,  0.0963, -0.2658,
          -0.1464],
         [ 0.1496, -0.3596, -0.1726, -0.1457, -0.0202, -0.1257, -0.0626,
          -0.2295, -0.3345,  0.2137, -0.2111,  0.0920, -0.1474,  0.2324,
          -0.0257, -0.0202,  0.3253, -0.1405,  0.1100, -0.0922, -0.1377,
           0.0540,  0.2066, -0.0022, -0.0961, -0.3093, -0.2066, -0.0564,
           0.2395,  0.1950, -0.

In [36]:
def explain_hidden_dimension():
    """
    详细解释隐藏维度的概念和作用
    """
    print("=" * 60)
    print("隐藏维度（Hidden Dimension）详解")
    print("=" * 60)
    
    # 1. 基本概念演示
    print("1. 基本概念：")
    print("   隐藏维度 = 每个词/token 的特征向量的长度")
    print("   更大的隐藏维度 = 更丰富的表示能力")
    print()
    
    # 创建不同隐藏维度的例子
    batch_size = 1
    seq_len = 4  # 4个词的句子
    
    # 小隐藏维度
    small_dim = 8
    small_features = torch.randn(batch_size, seq_len, small_dim)
    
    # 大隐藏维度
    large_dim = 512
    large_features = torch.randn(batch_size, seq_len, large_dim)
    
    print("2. 不同隐藏维度的对比：")
    print(f"   小隐藏维度 (dim={small_dim}): {small_features.shape}")
    print(f"   大隐藏维度 (dim={large_dim}): {large_features.shape}")
    print()
    print("   解释：")
    print(f"   - 小维度：每个词用 {small_dim} 个数字表示")
    print(f"   - 大维度：每个词用 {large_dim} 个数字表示")
    print("   - 更多数字 = 更丰富的语义表示")
    print()
    
    # 3. 隐藏维度与多头注意力的关系
    print("3. 隐藏维度与多头注意力的关系：")
    print("   核心原理：隐藏维度必须能被头数整除")
    print()
    
    # 演示不同的配置
    configs = [
        (64, 8),    # dim=64, n_heads=8
        (512, 8),   # dim=512, n_heads=8
        (512, 16),  # dim=512, n_heads=16
    ]
    
    for dim, n_heads in configs:
        head_dim = dim // n_heads
        print(f"   配置: dim={dim}, n_heads={n_heads}")
        print(f"   -> 每个头的维度 = {dim} ÷ {n_heads} = {head_dim}")
        print(f"   -> 意义：将 {dim} 维特征分成 {n_heads} 个 {head_dim} 维的子空间")
        print()
    
    return small_features, large_features

explain_hidden_dimension()


隐藏维度（Hidden Dimension）详解
1. 基本概念：
   隐藏维度 = 每个词/token 的特征向量的长度
   更大的隐藏维度 = 更丰富的表示能力

2. 不同隐藏维度的对比：
   小隐藏维度 (dim=8): torch.Size([1, 4, 8])
   大隐藏维度 (dim=512): torch.Size([1, 4, 512])

   解释：
   - 小维度：每个词用 8 个数字表示
   - 大维度：每个词用 512 个数字表示
   - 更多数字 = 更丰富的语义表示

3. 隐藏维度与多头注意力的关系：
   核心原理：隐藏维度必须能被头数整除

   配置: dim=64, n_heads=8
   -> 每个头的维度 = 64 ÷ 8 = 8
   -> 意义：将 64 维特征分成 8 个 8 维的子空间

   配置: dim=512, n_heads=8
   -> 每个头的维度 = 512 ÷ 8 = 64
   -> 意义：将 512 维特征分成 8 个 64 维的子空间

   配置: dim=512, n_heads=16
   -> 每个头的维度 = 512 ÷ 16 = 32
   -> 意义：将 512 维特征分成 16 个 32 维的子空间



(tensor([[[ 0.4122,  0.5732, -0.3689, -0.6788, -1.4379,  1.1635, -0.6507,
            1.2775],
          [ 0.5263,  1.2405,  0.5946,  0.4822,  0.9842, -0.2387, -0.5439,
            0.8796],
          [ 1.4528,  0.1149, -0.1737, -0.0449, -0.4003,  0.5686, -0.1289,
           -0.4169],
          [-0.3138, -0.4388,  0.7239,  0.3780,  1.5564,  0.5695,  2.5011,
           -2.2259]]]),
 tensor([[[ 1.4372, -0.3388, -0.9061,  ..., -1.2761, -0.3965,  0.3438],
          [ 0.9279, -0.2351,  1.1933,  ...,  2.1610, -1.4266, -0.0313],
          [-0.5602,  0.5114,  0.1775,  ...,  0.2276, -2.6811,  0.0305],
          [-2.2060,  0.4727, -2.2430,  ...,  0.2253,  2.7395,  0.2348]]]))

In [ ]:
def demonstrate_dimension_splitting():
    """
    演示隐藏维度如何在多头注意力中被分割
    """
    print("=" * 60)
    print("隐藏维度分割过程详解")
    print("=" * 60)
    
    # 设置参数
    batch_size = 1
    seq_len = 3
    dim = 12  # 使用小的维度便于理解
    n_heads = 3
    head_dim = dim // n_heads
    
    print(f"配置参数：")
    print(f"  batch_size: {batch_size}")
    print(f"  seq_len: {seq_len} (3个词)")
    print(f"  dim: {dim} (隐藏维度)")
    print(f"  n_heads: {n_heads} (注意力头数)")
    print(f"  head_dim: {head_dim} (每个头的维度)")
    print()
    
    # 1. 创建输入数据
    x = torch.randn(batch_size, seq_len, dim)
    print("1. 输入数据：")
    print(f"   形状: {x.shape}")
    print(f"   含义: {batch_size} 个样本, {seq_len} 个词, 每个词 {dim} 维特征")
    print()
    
    # 2. 模拟线性变换 (Q, K, V)
    # 这里我们只展示Q的变换过程
    Wq = torch.randn(dim, dim)  # Q的权重矩阵
    
    # 计算Q
    Q = torch.matmul(x, Wq)  # (1, 3, 12) × (12, 12) = (1, 3, 12)
    print("2. 线性变换后的Q：")
    print(f"   Q形状: {Q.shape}")
    print(f"   这仍然是 {seq_len} 个词，每个词 {dim} 维特征")
    print()
    
    # 3. 重塑为多头形式
    Q_reshaped = Q.view(batch_size, seq_len, n_heads, head_dim)
    print("3. 重塑为多头形式：")
    print(f"   Q_reshaped形状: {Q_reshaped.shape}")
    print(f"   含义: {batch_size} 个样本, {seq_len} 个词, {n_heads} 个头, 每个头 {head_dim} 维")
    print()
    
    # 4. 详细查看分割结果
    print("4. 详细查看每个头的数据：")
    for head in range(n_heads):
        head_data = Q_reshaped[0, :, head, :]  # 第一个样本，所有词，第head个头
        print(f"   头 {head}: 形状 {head_data.shape}")
        print(f"   -> 包含 {seq_len} 个词，每个词 {head_dim} 维特征")
        print(f"   -> 原始特征的第 {head*head_dim} 到第 {(head+1)*head_dim-1} 维")
        print()
    
    # 5. 转置准备计算
    Q_transposed = Q_reshaped.transpose(1, 2)
    print("5. 转置后用于注意力计算：")
    print(f"   Q_transposed形状: {Q_transposed.shape}")
    print(f"   含义: {batch_size} 个样本, {n_heads} 个头, {seq_len} 个词, 每个头 {head_dim} 维")
    print()
    
    # 6. 展示原始特征如何被分割
    print("6. 原始特征分割示例：")
    print("   假设原始12维特征表示：[语法, 语义, 位置, 情感, 主题, 语调, 时态, 语态, 词性, 依赖, 共指, 语音]")
    print("   分割后：")
    print("   - 头0: [语法, 语义, 位置, 情感] (维度 0-3)")
    print("   - 头1: [主题, 语调, 时态, 语态] (维度 4-7)")
    print("   - 头2: [词性, 依赖, 共指, 语音] (维度 8-11)")
    print("   每个头专注于不同的语言特征！")
    print()
    
    return Q_reshaped, Q_transposed

demonstrate_dimension_splitting()


In [ ]:
def compare_hidden_dimensions():
    """
    比较不同隐藏维度对模型的影响
    """
    print("=" * 60)
    print("不同隐藏维度的影响对比")
    print("=" * 60)
    
    # 测试不同的隐藏维度配置
    configs = [
        {"dim": 64, "n_heads": 4, "name": "小型模型"},
        {"dim": 256, "n_heads": 8, "name": "中型模型"},
        {"dim": 512, "n_heads": 8, "name": "大型模型"},
    ]
    
    batch_size = 1
    seq_len = 5
    
    print("测试配置：")
    print(f"  输入: batch_size={batch_size}, seq_len={seq_len}")
    print()
    
    for i, config in enumerate(configs):
        print(f"{i+1}. {config['name']}:")
        print(f"   隐藏维度: {config['dim']}")
        print(f"   注意力头数: {config['n_heads']}")
        print(f"   每个头维度: {config['dim'] // config['n_heads']}")
        
        # 计算参数数量
        dim = config['dim']
        n_heads = config['n_heads']
        
        # Q, K, V 线性层参数
        qkv_params = 3 * dim * dim  # 3个线性层，每个 dim×dim
        # 输出线性层参数
        output_params = dim * dim
        total_params = qkv_params + output_params
        
        print(f"   参数数量: {total_params:,}")
        print(f"   内存占用: ~{total_params * 4 / 1024:.1f} KB (float32)")
        
        # 创建模型
        args = ModelArgs(dim=dim, n_heads=n_heads, dropout=0.0)
        model = MultiHeadAttention(args, is_causal=False)
        
        # 计算输出
        x = torch.randn(batch_size, seq_len, dim)
        with torch.no_grad():
            output = model(x, x, x)
        
        print(f"   输入形状: {x.shape}")
        print(f"   输出形状: {output.shape}")
        print(f"   表示能力: {dim}维向量可以表示 2^{dim} 种不同的模式")
        print()
    
    # 展示隐藏维度选择的权衡
    print("=" * 40)
    print("隐藏维度选择的权衡")
    print("=" * 40)
    
    print("🔺 更大的隐藏维度 (优点):")
    print("  ✓ 更强的表示能力")
    print("  ✓ 能捕获更复杂的语言模式")
    print("  ✓ 更好的性能表现")
    print("  ✓ 更大的模型容量")
    print()
    
    print("🔻 更大的隐藏维度 (缺点):")
    print("  ✗ 需要更多内存")
    print("  ✗ 计算成本更高")
    print("  ✗ 容易过拟合")
    print("  ✗ 训练时间更长")
    print()
    
    print("🎯 实际应用中的选择:")
    print("  • GPT-2 small: dim=768")
    print("  • GPT-2 medium: dim=1024")
    print("  • GPT-2 large: dim=1280")
    print("  • GPT-3: dim=12288")
    print()
    
    print("💡 选择建议:")
    print("  1. 从小模型开始，逐步增加维度")
    print("  2. 根据任务复杂度选择合适的维度")
    print("  3. 考虑计算资源和时间限制")
    print("  4. 使用验证集来选择最优维度")

compare_hidden_dimensions()


In [ ]:
def hidden_dimension_complete_flow():
    """
    展示隐藏维度在多头注意力中的完整流程
    """
    print("=" * 70)
    print("隐藏维度在多头注意力中的完整流程")
    print("=" * 70)
    
    # 使用容易理解的小参数
    dim = 16
    n_heads = 4
    head_dim = dim // n_heads
    seq_len = 3
    
    print("📋 流程概览:")
    print(f"   原始输入: 每个词 {dim} 维特征")
    print(f"   分割: 分成 {n_heads} 个头，每个头 {head_dim} 维")
    print(f"   计算: 每个头独立计算注意力")
    print(f"   合并: 将 {n_heads} 个头的结果合并回 {dim} 维")
    print()
    
    # 创建输入数据
    x = torch.randn(1, seq_len, dim)
    print(f"🔸 步骤1: 输入数据")
    print(f"   形状: {x.shape}")
    print(f"   含义: 3个词，每个词16维特征向量")
    print()
    
    # 模拟权重矩阵
    W_q = torch.randn(dim, dim)
    W_k = torch.randn(dim, dim)
    W_v = torch.randn(dim, dim)
    W_o = torch.randn(dim, dim)
    
    print(f"🔸 步骤2: 线性变换 (Q, K, V)")
    Q = torch.matmul(x, W_q)
    K = torch.matmul(x, W_k)
    V = torch.matmul(x, W_v)
    print(f"   Q形状: {Q.shape} (查询)")
    print(f"   K形状: {K.shape} (键)")
    print(f"   V形状: {V.shape} (值)")
    print()
    
    print(f"🔸 步骤3: 重塑为多头形式")
    Q_multi = Q.view(1, seq_len, n_heads, head_dim).transpose(1, 2)
    K_multi = K.view(1, seq_len, n_heads, head_dim).transpose(1, 2)
    V_multi = V.view(1, seq_len, n_heads, head_dim).transpose(1, 2)
    print(f"   Q_multi形状: {Q_multi.shape}")
    print(f"   含义: 1个样本, {n_heads}个头, {seq_len}个词, 每个头{head_dim}维")
    print()
    
    print(f"🔸 步骤4: 每个头独立计算注意力")
    attention_outputs = []
    for head in range(n_heads):
        q_head = Q_multi[:, head, :, :]  # (1, 3, 4)
        k_head = K_multi[:, head, :, :]  # (1, 3, 4)
        v_head = V_multi[:, head, :, :]  # (1, 3, 4)
        
        # 计算注意力分数
        scores = torch.matmul(q_head, k_head.transpose(-2, -1)) / math.sqrt(head_dim)
        attn_weights = F.softmax(scores, dim=-1)
        attn_output = torch.matmul(attn_weights, v_head)
        
        attention_outputs.append(attn_output)
        print(f"   头{head}: {q_head.shape} -> {attn_output.shape}")
    
    print(f"   每个头都产生了 {seq_len} 个词的 {head_dim} 维表示")
    print()
    
    print(f"🔸 步骤5: 合并多头结果")
    # 将所有头的输出合并
    multi_head_output = torch.stack(attention_outputs, dim=1)  # (1, 4, 3, 4)
    print(f"   合并后形状: {multi_head_output.shape}")
    
    # 重塑回原始形式
    merged_output = multi_head_output.transpose(1, 2).contiguous().view(1, seq_len, dim)
    print(f"   重塑后形状: {merged_output.shape}")
    print(f"   含义: 恢复到 {seq_len} 个词，每个词 {dim} 维特征")
    print()
    
    print(f"🔸 步骤6: 最终线性变换")
    final_output = torch.matmul(merged_output, W_o)
    print(f"   最终输出形状: {final_output.shape}")
    print(f"   这与输入形状完全一致！")
    print()
    
    print("=" * 50)
    print("🎯 隐藏维度的关键作用总结")
    print("=" * 50)
    
    print("1. 📊 信息容量:")
    print(f"   • {dim}维向量可以编码丰富的语言信息")
    print(f"   • 更大的维度 = 更强的表示能力")
    print()
    
    print("2. 🔄 并行处理:")
    print(f"   • 将{dim}维特征分割成{n_heads}个{head_dim}维子空间")
    print(f"   • 每个头专注于不同的语言特征")
    print(f"   • 并行计算提高效率")
    print()
    
    print("3. 🎨 特征多样性:")
    print("   • 不同的头可能学习：")
    print("     - 头0: 语法关系 (主谓宾)")
    print("     - 头1: 语义相似性")
    print("     - 头2: 位置信息")
    print("     - 头3: 情感倾向")
    print()
    
    print("4. 🔧 设计约束:")
    print(f"   • dim 必须能被 n_heads 整除")
    print(f"   • 当前: {dim} ÷ {n_heads} = {head_dim} ✓")
    print(f"   • 这确保了均匀的特征分配")
    print()
    
    print("5. 💡 实际意义:")
    print("   • 输入输出维度保持一致")
    print("   • 可以叠加多个注意力层")
    print("   • 兼容残差连接和层归一化")
    
    return final_output

hidden_dimension_complete_flow()


In [ ]:
def visualize_attention_weights():
    """
    可视化注意力权重，展示因果掩码的效果
    """
    print("=" * 60)
    print("注意力权重可视化演示")
    print("=" * 60)
    
    # 创建一个简单的多头注意力模块用于可视化
    class SimpleMultiHeadAttention(nn.Module):
        def __init__(self, dim, n_heads, is_causal=False):
            super().__init__()
            self.dim = dim
            self.n_heads = n_heads
            self.head_dim = dim // n_heads
            self.is_causal = is_causal
            
            self.wq = nn.Linear(dim, dim, bias=False)
            self.wk = nn.Linear(dim, dim, bias=False)
            self.wv = nn.Linear(dim, dim, bias=False)
            self.wo = nn.Linear(dim, dim, bias=False)
            
            if is_causal:
                # 创建因果掩码
                max_len = 10
                mask = torch.full((max_len, max_len), float('-inf'))
                mask = torch.triu(mask, diagonal=1)
                self.register_buffer('mask', mask)
        
        def forward(self, x):
            B, T, C = x.shape
            
            # 计算 Q, K, V
            q = self.wq(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
            k = self.wk(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
            v = self.wv(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
            
            # 计算注意力分数
            scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
            
            # 应用因果掩码
            if self.is_causal:
                scores = scores + self.mask[:T, :T]
            
            # 计算注意力权重
            attn_weights = F.softmax(scores, dim=-1)
            
            # 应用注意力权重
            out = torch.matmul(attn_weights, v)
            out = out.transpose(1, 2).contiguous().view(B, T, C)
            out = self.wo(out)
            
            return out, attn_weights
    
    # 创建两个模型：普通的和因果的
    dim = 32
    n_heads = 2
    seq_len = 5
    
    model_normal = SimpleMultiHeadAttention(dim, n_heads, is_causal=False)
    model_causal = SimpleMultiHeadAttention(dim, n_heads, is_causal=True)
    
    # 创建输入数据
    x = torch.randn(1, seq_len, dim)
    
    print(f"输入数据形状: {x.shape}")
    print(f"模型配置: dim={dim}, n_heads={n_heads}, seq_len={seq_len}")
    print()
    
    # 运行两个模型
    with torch.no_grad():
        output_normal, attn_normal = model_normal(x)
        output_causal, attn_causal = model_causal(x)
    
    print("普通多头注意力权重 (第一个头):")
    print("行：查询位置，列：键位置")
    print("数值表示每个查询位置对每个键位置的注意力权重")
    print("-" * 50)
    normal_weights = attn_normal[0, 0].numpy()  # 第一个样本，第一个头
    for i in range(seq_len):
        row_str = " ".join([f"{val:.3f}" for val in normal_weights[i]])
        print(f"位置 {i}: [{row_str}]")
    print()
    
    print("因果多头注意力权重 (第一个头):")
    print("注意：上三角部分应该全为0（或很小），因为不能看到未来信息")
    print("-" * 50)
    causal_weights = attn_causal[0, 0].numpy()  # 第一个样本，第一个头
    for i in range(seq_len):
        row_str = " ".join([f"{val:.3f}" for val in causal_weights[i]])
        print(f"位置 {i}: [{row_str}]")
    print()
    
    # 验证因果掩码的效果
    print("验证因果掩码效果:")
    print("-" * 30)
    
    # 检查上三角部分是否为0
    upper_triangle_sum = 0
    for i in range(seq_len):
        for j in range(i+1, seq_len):
            upper_triangle_sum += causal_weights[i, j]
    
    print(f"因果注意力权重上三角部分的和: {upper_triangle_sum:.6f}")
    print("(应该接近0，表示未来信息被屏蔽)")
    
    # 检查每行的和是否为1
    print("\\n每行注意力权重的和 (应该都约等于1):")
    for i in range(seq_len):
        row_sum = causal_weights[i].sum()
        print(f"  位置 {i}: {row_sum:.6f}")
    
    return attn_normal, attn_causal

# 运行可视化演示
visualize_attention_weights()


In [25]:
#define the ModelArgs Type
from dataclasses import dataclass

@dataclass
class ModelArgs:
    """模型配置参数类"""
    dim: int = 512                    # 模型维度
    n_heads: int = 8                  # 注意力头数
    n_layers: int = 6                 # 层数
    vocab_size: int = 50000           # 词汇表大小
    max_seq_len: int = 2048           # 最大序列长度
    dropout: float = 0.1              # Dropout 比率
    bias: bool = False                # 是否使用偏置


'''多头自注意力计算模块'''
class MultiHeadAttention(nn.Module):

    def __init__(self, args: ModelArgs, is_causal=False):
        # 构造函数
        # args: 配置对象
        super().__init__()
        # 隐藏层维度必须是头数的整数倍，因为后面我们会将输入拆成头数个矩阵
        assert args.dim % args.n_heads == 0
        # 模型并行处理大小，默认为1。
        model_parallel_size = 1
        # 本地计算头数，等于总头数除以模型并行处理大小。
        self.n_local_heads = args.n_heads // model_parallel_size
        # 每个头的维度，等于模型维度除以头的总数。
        self.head_dim = args.dim // args.n_heads

        # Wq, Wk, Wv 参数矩阵，每个参数矩阵为 n_embd x n_embd
        # 这里通过三个组合矩阵来代替了n个参数矩阵的组合，其逻辑在于矩阵内积再拼接其实等同于拼接矩阵再内积，
        # 不理解的读者可以自行模拟一下，每一个线性层其实相当于n个参数矩阵的拼接
        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wk = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wv = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        # 输出权重矩阵，维度为 dim x n_embd（head_dim = n_embeds / n_heads）
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)
        # 注意力的 dropout
        self.attn_dropout = nn.Dropout(args.dropout)
        # 残差连接的 dropout
        self.resid_dropout = nn.Dropout(args.dropout)
         
        # 创建一个上三角矩阵，用于遮蔽未来信息
        # 注意，因为是多头注意力，Mask 矩阵比之前我们定义的多一个维度
        if is_causal:
           mask = torch.full((1, 1, args.max_seq_len, args.max_seq_len), float("-inf"))
           mask = torch.triu(mask, diagonal=1)
           # 注册为模型的缓冲区
           self.register_buffer("mask", mask)

    def forward(self, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor):

        # 获取批次大小和序列长度，[batch_size, seq_len, dim]
        bsz, seqlen, _ = q.shape

        # 计算查询（Q）、键（K）、值（V）,输入通过参数矩阵层，维度为 (B, T, n_embed) x (n_embed, n_embed) -> (B, T, n_embed)
        xq, xk, xv = self.wq(q), self.wk(k), self.wv(v)

        # 将 Q、K、V 拆分成多头，维度为 (B, T, n_head, C // n_head)，然后交换维度，变成 (B, n_head, T, C // n_head)
        # 因为在注意力计算中我们是取了后两个维度参与计算
        # 为什么要先按B*T*n_head*C//n_head展开再互换1、2维度而不是直接按注意力输入展开，是因为view的展开方式是直接把输入全部排开，
        # 然后按要求构造，可以发现只有上述操作能够实现我们将每个头对应部分取出来的目标
        xq = xq.view(bsz, seqlen, self.n_local_heads, self.head_dim)
        xk = xk.view(bsz, seqlen, self.n_local_heads, self.head_dim)
        xv = xv.view(bsz, seqlen, self.n_local_heads, self.head_dim)
        xq = xq.transpose(1, 2)
        xk = xk.transpose(1, 2)
        xv = xv.transpose(1, 2)


        # 注意力计算
        # 计算 QK^T / sqrt(d_k)，维度为 (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        scores = torch.matmul(xq, xk.transpose(2, 3)) / math.sqrt(self.head_dim)
        # 掩码自注意力必须有注意力掩码
        if self.is_causal:
            assert hasattr(self, 'mask')
            # 这里截取到序列长度，因为有些序列可能比 max_seq_len 短
            scores = scores + self.mask[:, :, :seqlen, :seqlen]
        # 计算 softmax，维度为 (B, nh, T, T)
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)
        # 做 Dropout
        scores = self.attn_dropout(scores)
        # V * Score，维度为(B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        output = torch.matmul(scores, xv)

        # 恢复时间维度并合并头。
        # 将多头的结果拼接起来, 先交换维度为 (B, T, n_head, C // n_head)，再拼接成 (B, T, n_head * C // n_head)
        # contiguous 函数用于重新开辟一块新内存存储，因为Pytorch设置先transpose再view会报错，
        # 因为view直接基于底层存储得到，然而transpose并不会改变底层存储，因此需要额外存储
        output = output.transpose(1, 2).contiguous().view(bsz, seqlen, -1)

        # 最终投影回残差流。
        output = self.wo(output)
        output = self.resid_dropout(output)
        return output

NameError: name 'ModelArgs' is not defined